<a href="https://colab.research.google.com/github/xcellentbird/Machine_Learning/blob/main/3%EB%B6%84_torch_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [132]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets, models

In [133]:
CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if CUDA else 'cpu')
print(DEVICE)

cuda


In [134]:
epochs = 300
batch_size = 128

In [135]:
Normalizer = transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
train_data_transform = transforms.Compose([
                                           transforms.RandomCrop(32, padding=4),
                                           transforms.RandomHorizontalFlip(),
                                           transforms.ToTensor(),
                                           Normalizer
])
test_data_transform = transforms.Compose([
                                          transforms.ToTensor(),
                                          Normalizer
])

In [136]:
train_datasets = datasets.CIFAR10('./', train=True, download=True, transform=train_data_transform)
test_datasets = datasets.CIFAR10('./', train=False, transform=test_data_transform)

Files already downloaded and verified


In [137]:
train_loader = torch.utils.data.DataLoader(train_datasets, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_datasets, batch_size=batch_size, shuffle=False)

In [192]:
class BasicBlock(nn.Module):
  def __init__(self, in_planes, planes, stride=1):
    super(BasicBlock, self).__init__()
    # 정상 루트
    self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3,
                           stride=stride, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(planes)
    self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                           stride=1, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(planes)
    
    # 지름길 루트
    # 입력값이 정상 루트를 거치면서 크기가 변한다면,
    # 지름길 루트도 똑같이 크기를 변환
    self.shortcut = nn.Sequential()
    if stride != 1 or in_planes != planes:
      self.shortcut = nn.Sequential(
          nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
          nn.BatchNorm2d(planes)
        )

  def forward(self, x):
    # 정상 루트
    out = self.conv1(x)
    out = self.bn1(out)
    out = F.relu(out)
    out = self.conv2(out)
    out = self.bn2(out)

    # 정상 루트와 지름길 루트 결과값을 더한다.
    out += self.shortcut(x)
    out = F.relu(out)
    return out

In [193]:
class ResNet(nn.Module):
  def __init__(self, num_classes = 10):
    super(ResNet, self).__init__()
    self.in_planes = 16

    self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(16)
    self.layer1 = self._make_layer(16, 2, stride=1)
    self.layer2 = self._make_layer(32, 2, stride=2)
    self.layer3 = self._make_layer(64, 2, stride=2)
    self.linear = nn.Linear(64, num_classes)

  def _make_layer(self, planes, num_blocks, stride):
    strides = [stride] + [1]*(num_blocks-1)
    layers = []
    for stride in strides:
      layers.append(BasicBlock(self.in_planes, planes, stride))
      self.in_planes = planes
    return nn.Sequential(*layers)

  def forward(self, x):
    out = self.conv1(x)
    out = self.bn1(out)
    out = F.relu(out)
    out = self.layer1(out)
    out = self.layer2(out)
    out = self.layer3(out)
    out = F.avg_pool2d(out, 8)
    out = out.view(out.size(0), -1)
    out = self.linear(out)
    return out

In [194]:
model = ResNet().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=0.0005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

In [195]:
def train(model, train_loader, optimizer, epoch):
  model.train()
  for idx, (data, target) in enumerate(train_loader):
    data, target = data.to(DEVICE), target.to(DEVICE)
    optimizer.zero_grad()
    output = model(data)
    loss = F.cross_entropy(output, target)
    loss.backward()
    optimizer.step()

In [196]:
def evaluate(model, test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(DEVICE), target.to(DEVICE)
      output = model(data)

      test_loss += F.cross_entropy(output, target, reduction='sum').item()

      prob, pred = output.max(1, keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)
  test_accuracy = 100. * correct / len(test_loader.dataset)
  return test_loss, test_accuracy

In [197]:
for epoch in range(epochs):
  train(model, train_loader, optimizer, epoch+1)
  test_loss, test_accuracy = evaluate(model, test_loader)
  
  print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch+1, test_loss, test_accuracy))

  scheduler.step()

[1] Test Loss: 1.4996, Accuracy: 46.78%
[2] Test Loss: 1.3402, Accuracy: 55.13%
[3] Test Loss: 1.1250, Accuracy: 61.63%
[4] Test Loss: 0.8831, Accuracy: 68.31%
[5] Test Loss: 0.8326, Accuracy: 71.17%
[6] Test Loss: 0.8082, Accuracy: 73.13%
[7] Test Loss: 0.8800, Accuracy: 71.68%
[8] Test Loss: 0.8522, Accuracy: 72.91%
[9] Test Loss: 0.7987, Accuracy: 72.69%
[10] Test Loss: 0.7695, Accuracy: 74.37%
[11] Test Loss: 0.7744, Accuracy: 75.43%
[12] Test Loss: 0.8132, Accuracy: 74.50%
[13] Test Loss: 0.7101, Accuracy: 76.06%
[14] Test Loss: 0.7883, Accuracy: 74.33%
[15] Test Loss: 0.7531, Accuracy: 75.54%
[16] Test Loss: 0.6600, Accuracy: 77.48%
[17] Test Loss: 0.6831, Accuracy: 77.35%
[18] Test Loss: 0.6473, Accuracy: 78.63%
[19] Test Loss: 0.7923, Accuracy: 73.18%
[20] Test Loss: 0.8168, Accuracy: 74.49%
[21] Test Loss: 0.7149, Accuracy: 76.11%
[22] Test Loss: 0.8015, Accuracy: 73.68%
[23] Test Loss: 0.8945, Accuracy: 73.12%
[24] Test Loss: 0.6868, Accuracy: 78.15%
[25] Test Loss: 0.8050, A